## VINCENT CORPES JR. | BSCS 3A   | CCS 246   |  EXERCISE FOR UNIT 2

# 1. Choose either one of the following tasks (the output of this task will be used on the next number):
a.	Develop a Class in Python called Dense_Layer (included in the submitted notebook).

The chosen task should have the following functions:
    a. (10 points) A function to setup/accept the inputs and weights
    b. (10 points) A function to perform the weighted sum + bias
    c. (15 points) A function to perform the selected activation function
    d. (15 points) A function to calculate the loss (predicted output vs target output)



In [8]:
import numpy as np

class Dense_Layer:
    def __init__(self, W=None, b=None, activation='linear', name='Dense'):
        self.W = None if W is None else np.array(W, dtype=float)
        self.b = None if b is None else np.array(b, dtype=float)
        self.activation_name = activation
        self.name = name
        self.X = None
        self.z = None
        self.a = None

    def set_params(self, W, b):
        self.W = np.array(W, dtype=float)
        self.b = np.array(b, dtype=float)

    def set_input(self, X):
        self.X = np.array(X, dtype=float)

    def weighted_sum(self):
        self.z = self.W.dot(self.X) + self.b
        return self.z

    def activate(self, z=None, func=None):
        if z is None:
            z = self.z
        if func is None:
            func = self.activation_name
        func = func.lower()

        if func == 'relu':
            self.a = np.maximum(0, z)
        elif func == 'sigmoid':
            self.a = 1 / (1 + np.exp(-z))
        elif func == 'softmax':
            exp_z = np.exp(z - np.max(z))  # for numerical stability
            self.a = exp_z / np.sum(exp_z)
        elif func == 'linear':
            self.a = z
        else:
            raise ValueError("Unsupported activation")
        return self.a

    @staticmethod
    def compute_loss(y_pred, y_true, loss='mse'):
        y_pred = np.array(y_pred, dtype=float)
        y_true = np.array(y_true, dtype=float)
        if loss == 'mse':
            return np.mean((y_pred - y_true)**2)
        elif loss in ('bce', 'binary_crossentropy'):
            eps = 1e-12
            y_pred = np.clip(y_pred, eps, 1 - eps)
            return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))
        elif loss in ('ce', 'categorical_crossentropy'):
            eps = 1e-12
            y_pred = np.clip(y_pred, eps, 1 - eps)
            return -np.sum(y_true * np.log(y_pred))
        else:
            raise ValueError("Unsupported loss type")

def detailed_dot_print(W_row, X, bias, neuron_idx=None):
    terms = W_row * X
    sum_terms = terms.sum()
    z = sum_terms + bias
    label = f"Neuron {neuron_idx}: " if neuron_idx else ""
    print(f"{label}{' + '.join([f'({w}*{x})' for w,x in zip(W_row, X)])} + ({bias}) = {z:.10f}")
    return z


a. Given the following inputs from the Iris Dataset, using the sepal length, sepal width, petal length and petal width, determine what class (Iris-setosa, Iris-versicolor, and Iris-virginica) the following inputs are by calculating the output, given the neural network configurations:

In [12]:
X = np.array([5.1, 3.5, 1.4, 0.2])
target = np.array([0.7, 0.2, 0.1])  

W1 = np.array([[0.2,  0.5, -0.3, 0.1],
               [-0.2, 0.4, -0.4, 0.3],
               [0.2,  0.6, -0.1, 0.5]])
b1 = np.array([3.0, -2.1, 0.6])

W2 = np.array([[0.3, -0.5, 0.7],
               [0.2, -0.6, 0.4]])
b2 = np.array([4.3, 6.4])

W3 = np.array([[0.5, -0.3],
               [0.8, -0.2],
               [0.6, -0.4]])
b3 = np.array([-1.5, 2.1, -3.3])

print("Input X:", X)

# Hidden Layer 1
layer1 = Dense_Layer(W1, b1, activation='relu')
layer1.set_input(X)
z1 = layer1.weighted_sum()
print("\n-hIDDEN LAYER 1")
for i in range(W1.shape[0]):
    detailed_dot_print(W1[i], X, b1[i], neuron_idx=i+1)
a1 = layer1.activate()
print("ReLU(z1) =", np.round(a1, 10))

# Hidden Layer 2
layer2 = Dense_Layer(W2, b2, activation='sigmoid')
layer2.set_input(a1)
z2 = layer2.weighted_sum()
print("\nHIDDEN LAEER 2")
for i in range(W2.shape[0]):
    detailed_dot_print(W2[i], a1, b2[i], neuron_idx=i+1)
a2 = layer2.activate()
print("Sigmoid(z2) =", np.round(a2, 10))

# Output Layer (Softmax)
layer3 = Dense_Layer(W3, b3, activation='softmax')
layer3.set_input(a2)
z3 = layer3.weighted_sum()
print("\nOUTPUT LAYER")
for i in range(W3.shape[0]):
    detailed_dot_print(W3[i], a2, b3[i], neuron_idx=i+1)
a3 = layer3.activate()
print("Softmax(z3) =", np.round(a3, 10))

# Loss calculation
ce_loss = Dense_Layer.compute_loss(a3, target, loss='ce')
predicted_class = np.argmax(a3)

print("\nLOSS")
print(f"Categorical Cross-Entropy Loss: {ce_loss:.10f}")
print(f"Predicted class index: {predicted_class} → "
      f"{['Iris-setosa','Iris-versicolor','Iris-virginica'][predicted_class]}")


Input X: [5.1 3.5 1.4 0.2]

-hIDDEN LAYER 1
Neuron 1: (0.2*5.1) + (0.5*3.5) + (-0.3*1.4) + (0.1*0.2) + (3.0) = 5.3700000000
Neuron 2: (-0.2*5.1) + (0.4*3.5) + (-0.4*1.4) + (0.3*0.2) + (-2.1) = -2.2200000000
Neuron 3: (0.2*5.1) + (0.6*3.5) + (-0.1*1.4) + (0.5*0.2) + (0.6) = 3.6800000000
ReLU(z1) = [5.37 0.   3.68]

HIDDEN LAEER 2
Neuron 1: (0.3*5.37) + (-0.5*0.0) + (0.7*3.68) + (4.3) = 8.4870000000
Neuron 2: (0.2*5.37) + (-0.6*0.0) + (0.4*3.68) + (6.4) = 8.9460000000
Sigmoid(z2) = [0.99979391 0.99986976]

OUTPUT LAYER
Neuron 1: (0.5*0.9997939117554883) + (-0.3*0.9998697598167464) + (-1.5) = -1.3000639721
Neuron 2: (0.8*0.9997939117554883) + (-0.2*0.9998697598167464) + (2.1) = 2.6998611774
Neuron 3: (0.6*0.9997939117554883) + (-0.4*0.9998697598167464) + (-3.3) = -3.1000715569
Softmax(z3) = [0.01793421 0.97910131 0.00296448]

LOSS
Categorical Cross-Entropy Loss: 3.4010610373
Predicted class index: 1 → Iris-versicolor


b. Given the following inputs from the Breast Cancer Dataset, using three features: Mean Radius, Mean Texture, and Mean Smoothness, determine whether the tumor is Benign (0) or Malignant (1) by calculating the network outputs step by step, given the following neural network configuration:

In [11]:
X = np.array([14.1, 20.3, 0.095]) 
target = np.array([1.0])

# First Hidden Layer
W1 = np.array([[ 0.5, -0.3,  0.8],
               [ 0.2,  0.4, -0.6],
               [-0.7,  0.9,  0.1]])
b1 = np.array([0.3, -0.5, 0.6])

# Second Hidden Layer
W2 = np.array([[ 0.6, -0.2,  0.4],
               [-0.3,  0.5,  0.7]])
b2 = np.array([0.1, -0.8])

# Output Layer
W3 = np.array([[0.7, -0.5]])
b3 = np.array([0.2])

print("Input X:", X)

# Hidden Layer 1
layer1 = Dense_Layer(W1, b1, activation='relu')
layer1.set_input(X)
z1 = layer1.weighted_sum()
print("\nHIDDEN LAYER 1")
for i in range(W1.shape[0]):
    detailed_dot_print(W1[i], X, b1[i], neuron_idx=i+1)
a1 = layer1.activate()
print("ReLU(z1) =", np.round(a1, 10))

# Hidden Layer 2
layer2 = Dense_Layer(W2, b2, activation='sigmoid')
layer2.set_input(a1)
z2 = layer2.weighted_sum()
print("\nHIDDEN LAYER 2")
for i in range(W2.shape[0]):
    detailed_dot_print(W2[i], a1, b2[i], neuron_idx=i+1)
a2 = layer2.activate()
print("Sigmoid(z2) =", np.round(a2, 10))

# Output Laye
layer3 = Dense_Layer(W3, b3, activation='sigmoid')
layer3.set_input(a2)
z3 = layer3.weighted_sum()
print("\nOUTPUT LAYER")
detailed_dot_print(W3[0], a2, b3[0], neuron_idx=1)
a3 = layer3.activate()
print("Sigmoid(z3) =", round(a3[0], 10))

# Loss Clcculation
mse_loss = Dense_Layer.compute_loss(a3, target, loss='mse')
bce_loss = Dense_Layer.compute_loss(a3, target, loss='bce')
predicted_class = int(a3 >= 0.5)

print("\nLOSS")
print(f"MSE Loss: {mse_loss:.10f}")
print(f"Binary Cross-Entropy Loss: {bce_loss:.10f}")
print(f"Predicted class: {predicted_class} (1 = Malignant, 0 = Benign)")


Input X: [14.1   20.3    0.095]

HIDDEN LAYER 1
Neuron 1: (0.5*14.1) + (-0.3*20.3) + (0.8*0.095) + (0.3) = 1.3360000000
Neuron 2: (0.2*14.1) + (0.4*20.3) + (-0.6*0.095) + (-0.5) = 10.3830000000
Neuron 3: (-0.7*14.1) + (0.9*20.3) + (0.1*0.095) + (0.6) = 9.0095000000
ReLU(z1) = [ 1.336  10.383   9.0095]

HIDDEN LAYER 2
Neuron 1: (0.6*1.336) + (-0.2*10.383000000000001) + (0.4*9.0095) + (0.1) = 2.4288000000
Neuron 2: (-0.3*1.336) + (0.5*10.383000000000001) + (0.7*9.0095) + (-0.8) = 10.2973500000
Sigmoid(z2) = [0.91899725 0.99996628]

OUTPUT LAYER
Neuron 1: (0.7*0.9189972481200018) + (-0.5*0.9999662787960715) + (0.2) = 0.3433149343
Sigmoid(z3) = 0.5849955347

LOSS
MSE Loss: 0.1722287062
Binary Cross-Entropy Loss: 0.5361510648
Predicted class: 1 (1 = Malignant, 0 = Benign)


C:\Users\corpe\AppData\Local\Temp\ipykernel_14840\2738801299.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(a3 >= 0.5)
